In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
import sys
from glob import glob
REPO_DIR = "/Users/mdorosan/Desktop/2024/elpha-singhealth-fh"
sys.path.append(REPO_DIR)
from datetime import datetime

import utils.PATHS as PATHS
import utils.utils as utils
# import utils.emr_utils as emr_utils
# import utils.load_utils as load_utils

/Users/mdorosan/miniconda3/envs/basic-ds-toolkit/lib/python3.12/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Notes

The following were observed in the exploration below:
* Potentially useful prescription-related columns:
    - `Generic Drug Name`
* Consider datetime columns as reference for how long medication has been going in relation to lipid-lowering treatment

In [3]:
dd_fp_list = glob(os.path.join(PATHS.DRUG_DISPENSED, "*.csv"))

In [35]:
test = pd.read_csv(dd_fp_list[0])
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165368 entries, 0 to 165367
Data columns (total 34 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   Institution Code           165368 non-null  object 
 1   Patient ID                 165368 non-null  object 
 2   Date of Birth              165368 non-null  object 
 3   Gender                     165368 non-null  object 
 4   Race                       165361 non-null  object 
 5   Country of Residence       1661 non-null    object 
 6   Resident Indicator         165363 non-null  object 
 7   Case No                    165368 non-null  object 
 8   Visit No                   165368 non-null  object 
 9   Drug Code                  3964 non-null    object 
 10  Drug Name                  165368 non-null  object 
 11  Drug Strength              144244 non-null  object 
 12  Drug Form                  159685 non-null  object 
 13  Generic Drug Id            15

/var/folders/8l/dm4zrq7953jcg1whmk8m0my80000gn/T/ipykernel_69537/3840776815.py:1: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  test = pd.read_csv(dd_fp_list[0])


## Questions

1. What to do when combinations within the lookback were observed?
2. The statin + ____ prescriptions should be addressed within lookback?

In [ ]:
# SPECIAL CORRECTION FACTORS
{
    "pravastatin sodium": ,  # 20MG
    "ezetimibe,simvastatin tab" : , # 10/10, 10/20
    "ezetimibe": , # 10mg
    "simvastatin": , # 10mg, 20mg
    "atorvastatin": , # 10mg, 40mg, 20mg
    "rosuvastatin": , # 10MG, 20MG


    "lovastatin": , # 20mg
    "metronidazole, nystatin": , # 500mg, 100,000 unit    
    "nystatin": , # 100,000IU
} 

In [36]:
test = test.dropna(subset=["Generic Drug Name"])

In [37]:
test["Generic Drug Name"] = test["Generic Drug Name"].str.lower()

In [60]:
drug_name = "nystatin"
# drug_name = "statin"
sub = test[test["Generic Drug Name"].str.startswith(drug_name)][["Generic Drug Name", "Drug Strength"]]
display(sub.head())
display(sub["Generic Drug Name"].unique())
display(sub["Drug Strength"].unique())

,Generic Drug Name,Drug Strength
7304,nystatin,"100,000IU/"
7305,nystatin,"100,000IU/"
12215,nystatin,"100,000IU/"
14138,nystatin,"100,000IU/"
15359,nystatin,"100,000IU/"


array(['nystatin'], dtype=object)

array(['100,000IU/'], dtype=object)

In [8]:
test[["Generic Drug Name", "Prescribed Quantity UOM", ""]]

,Generic Drug Name,Prescribed Quantity UOM,Case Status Description
0,Teriparatide,BOX,Expected Unknown
1,Aspirin,TAB,Expected Unknown
2,HydroxyCARBAmide,CAP,Expected Unknown
3,OMEprazole,CAP,Expected Unknown
4,MEMANTINE,TAB,Expected Unknown
...,...,...,...
165363,CeftriaXONE,AMPVIA,Expected Unknown
165364,Guaifenesin,BOT,Expected Unknown
165365,Lidocaine,AMPVIA,Expected Unknown
165366,Prednisolone,TAB,Expected Unknown


In [9]:
test[["Prescribed Quantity UOM", "Item Primary UOM", "Drug Strength", "Drug Form"]].head(20)

,Prescribed Quantity UOM,Item Primary UOM,Drug Strength,Drug Form
0,BOX,BOX,600MCG/2.4ML,INJ AMP&VIA
1,TAB,TAB,100mg,TAB
2,CAP,CAP,500mg,CAP
3,CAP,CAP,20MG,CAP
4,TAB,TAB,10MG,TAB
5,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN
7,TAB,TAB,1mg,TAB
8,TAB,TAB,3mg,TAB
9,TAB,TAB,3mg,TAB


In [22]:
test.iloc[:,-8:]

,Prescribed Quantity UOM,Admit Source Code,Admit Source Description,Case Status Code,Case Status Description,Case Type Code,Case Type Description,Nationality
0,BOX,EXPUNKNOWN,Expected Unknown,EXPUNKNOWN,Expected Unknown,O,Outpatient,NaN
1,TAB,EXPUNKNOWN,Expected Unknown,EXPUNKNOWN,Expected Unknown,O,Outpatient,NaN
2,CAP,EXPUNKNOWN,Expected Unknown,EXPUNKNOWN,Expected Unknown,O,Outpatient,NaN
3,CAP,EXPUNKNOWN,Expected Unknown,EXPUNKNOWN,Expected Unknown,O,Outpatient,NaN
4,TAB,EXPUNKNOWN,Expected Unknown,EXPUNKNOWN,Expected Unknown,O,Outpatient,NaN
...,...,...,...,...,...,...,...,...
165363,AMPVIA,EXPUNKNOWN,Expected Unknown,EXPUNKNOWN,Expected Unknown,O,Outpatient,NaN
165364,BOT,EXPUNKNOWN,Expected Unknown,EXPUNKNOWN,Expected Unknown,O,Outpatient,NaN
165365,AMPVIA,EXPUNKNOWN,Expected Unknown,EXPUNKNOWN,Expected Unknown,O,Outpatient,NaN
165366,TAB,EXPUNKNOWN,Expected Unknown,EXPUNKNOWN,Expected Unknown,O,Outpatient,NaN


In [13]:
test[["Add Date", "Start Date"]].head(200)

,Add Date,Start Date
0,2017-11-02 13:35:17,2017-11-02
1,2017-11-10 12:25:55,2017-11-10
2,2017-11-10 12:25:55,2017-11-10
3,2017-11-10 12:25:55,2017-11-10
4,2017-11-14 14:40:48,2017-11-14
...,...,...
195,2017-11-01 16:20:54,2017-11-01
196,2017-11-24 11:58:51,0-00-00
197,2017-11-24 11:58:49,0-00-00
198,2017-11-24 11:58:50,0-00-00


In [12]:
test[["Case Start Date", "Case End Date"]].head(200)

,Case Start Date,Case End Date
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
195,NaN,NaN
196,NaN,NaN
197,NaN,NaN
198,NaN,NaN


In [11]:
test[["Drug Prescribed Date From", "Drug Prescribed Date To"]].head(200)

,Drug Prescribed Date From,Drug Prescribed Date To
0,2017-11-02,2017-12-28
1,2017-11-10,2017-11-10
2,2017-11-10,2017-11-10
3,2017-11-10,2017-11-10
4,2017-11-14,2017-11-14
...,...,...
195,2017-11-01,2017-11-01
196,2017-11-24,2018-01-23
197,2017-11-24,2018-01-23
198,2017-11-24,2018-01-23


In [7]:
test[["Drug Code", "Drug Name", "Drug Strength", "Drug Form", "Generic Drug Id", "Generic Drug Name", "DRG Code", "DRG Code (AR-DRG V6.0)"]]

,Drug Code,Drug Name,Drug Strength,Drug Form,Generic Drug Id,Generic Drug Name,DRG Code,DRG Code (AR-DRG V6.0)
0,NaN,TERIPARATIDE 600MCG/2.4ML INJ (FORTEO),600MCG/2.4ML,INJ AMP&VIA,784,Teriparatide,Unknown,Unknown
1,NaN,Aspirin 100mg Tab,100mg,TAB,48,Aspirin,EXPUNKNOWN,EXPUNKNOWN
2,NaN,HydroxyCARBAmide [Hydroxyurea] 500mg Cap,500mg,CAP,323,HydroxyCARBAmide,EXPUNKNOWN,EXPUNKNOWN
3,NaN,OMEprazole 20MG CAPSULE,20MG,CAP,856,OMEprazole,EXPUNKNOWN,EXPUNKNOWN
4,NaN,MEMANTINE 10MG TAB,10MG,TAB,795,MEMANTINE,EXPUNKNOWN,EXPUNKNOWN
...,...,...,...,...,...,...,...,...
165363,NaN,CeftriaXONE 1g Inj,1g,INJ AMP&VIA,121,CeftriaXONE,Unknown,Unknown
165364,NaN,Guaifenesin 100mg/5mL Syrup,100mg/5mL,MIX,301,Guaifenesin,Unknown,Unknown
165365,NaN,Lidocaine (P/Free) 1% (200mg/20mL) Inj,1% 20mL,INJ AMP&VIA,367,Lidocaine,Unknown,Unknown
165366,NaN,Prednisolone 20mg Tab,20mg,TAB,522,Prednisolone,Unknown,Unknown


In [8]:
# "DRG Code", "DRG Code (AR-DRG V6.0)" # diagnosis related group
test["DRG Code"].value_counts()

DRG Code
EXPUNKNOWN    143861
Unknown        21507
Name: count, dtype: int64

In [9]:
test["DRG Code (AR-DRG V6.0)"].value_counts()

DRG Code (AR-DRG V6.0)
EXPUNKNOWN    143861
Unknown        21507
Name: count, dtype: int64

In [10]:
test["Drug Status"].value_counts()

Drug Status
STDTABCAP     60782
STDL2         40503
02            38144
STDCRMOINT     4628
STDMIX         4152
NF             1021
07              912
SVL             892
06              879
STDEEN          654
STDINSUL        641
STDINHAL        577
SVC             495
03              494
14              491
STDSOL1         479
05              435
STDINFINJ       420
STDITEM         415
08              348
22              322
STDSUPP         305
STDSOL4         270
STDL2ORG        226
STDSOL2         187
STDWEEK         141
04              133
20              126
19              104
13               82
10               78
70               44
CT               29
15               21
12               19
STDSOL3           4
60                1
Name: count, dtype: int64

In [4]:
test.head()

,Institution Code,Patient ID,Date of Birth,Gender,Race,Country of Residence,Resident Indicator,Case No,Visit No,Drug Code,...,Add Date,Start Date,Prescribed Quantity UOM,Admit Source Code,Admit Source Description,Case Status Code,Case Status Description,Case Type Code,Case Type Description,Nationality
0,SGH,12550cd5248c7f1a8ec1,1949-05-01,MALE,NaN,NaN,NaN,eb4d2d24ce205f326eff,eb4d2d24ce205f326eff,NaN,...,2017-11-02 13:35:17,2017-11-02,BOX,EXPUNKNOWN,Expected Unknown,EXPUNKNOWN,Expected Unknown,O,Outpatient,NaN
1,SGH,ca890aa7a9e150a3025c,1952-04-01,MALE,Chinese,NaN,N,db3c7a5a7a4b82b83a4d,58fcd47050aa15e1cc09,NaN,...,2017-11-10 12:25:55,2017-11-10,TAB,EXPUNKNOWN,Expected Unknown,EXPUNKNOWN,Expected Unknown,O,Outpatient,NaN
2,SGH,ca890aa7a9e150a3025c,1952-04-01,MALE,Chinese,NaN,N,db3c7a5a7a4b82b83a4d,58fcd47050aa15e1cc09,NaN,...,2017-11-10 12:25:55,2017-11-10,CAP,EXPUNKNOWN,Expected Unknown,EXPUNKNOWN,Expected Unknown,O,Outpatient,NaN
3,SGH,ca890aa7a9e150a3025c,1952-04-01,MALE,Chinese,NaN,N,db3c7a5a7a4b82b83a4d,58fcd47050aa15e1cc09,NaN,...,2017-11-10 12:25:55,2017-11-10,CAP,EXPUNKNOWN,Expected Unknown,EXPUNKNOWN,Expected Unknown,O,Outpatient,NaN
4,SGH,055ad3228632039dd19f,1934-09-01,MALE,Chinese,NaN,N,d82d3706cda79c5c8b58,91478522f25bbfca8d60,NaN,...,2017-11-14 14:40:48,2017-11-14,TAB,EXPUNKNOWN,Expected Unknown,EXPUNKNOWN,Expected Unknown,O,Outpatient,NaN
